

# Multi-class Classification for Predicting Stock Price Movement of Amazon using Neural Networks


### Import Packages

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import yfinance as yf
import pandas as pd

### Load Dataset

We read the data and map the textual categories (upwards, downwards and sideways) to numerical class labels (0, 1 or 2) and split of the dataset into training (80%) and test (20%) datasets. The input features are normalised to have zero mean and unit standard deviation.

In [2]:
# Download historical Amazon stock data
data = yf.download('AMZN', start='2010-01-01', end='2023-12-31')
print("Raw data shape:", data.shape)
data.head()

/var/folders/fm/3hy0z8052093y_1_c5sjdnc00000gn/T/ipykernel_26365/167139457.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('AMZN', start='2010-01-01', end='2023-12-31')
[*********************100%***********************]  1 of 1 completed

Raw data shape: (3522, 5)


Price,Close,High,Low,Open,Volume
Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
Date,,,,,
2010-01-04,6.6950,6.8305,6.6570,6.8125,151998000
2010-01-05,6.7345,6.7740,6.5905,6.6715,177038000
2010-01-06,6.6125,6.7365,6.5825,6.7300,143576000
2010-01-07,6.5000,6.6160,6.4400,6.6005,220604000
2010-01-08,6.6760,6.6840,6.4515,6.5280,196610000


In [3]:
# Feature Engineering
df = data.copy()

# 1. Daily Return (x1): (Close - Open) / Open
df['x1'] = (df['Close'] - df['Open']) / df['Open'] * 100  # as percentage

# 2. Intraday Volatility (x2): (High - Low) / Open
df['x2'] = (df['High'] - df['Low']) / df['Open'] * 100  # as percentage

# 3. Volume Signal (x3): Volume / 30-day average volume
df['x3'] = df['Volume'] / df['Volume'].rolling(window=30).mean()

# 4. Price Trend (x4): 10-day MA / 30-day MA (momentum indicator)
df['ma_10'] = df['Close'].rolling(window=10).mean()
df['ma_30'] = df['Close'].rolling(window=30).mean()
df['x4'] = df['ma_10'] / df['ma_30']

# Create target variable (next day's movement)
threshold = 1.0  # 1% threshold for meaningful movement
df['next_day_return'] = df['Close'].pct_change().shift(-1) * 100

df['y'] = 'sideways'  # default
df.loc[df['next_day_return'] > threshold, 'y'] = 'upwards'
df.loc[df['next_day_return'] < -threshold, 'y'] = 'downwards'

# Keep only the columns we need and drop NaN values
final_df = df[['x1', 'x2', 'x3', 'x4', 'y']].copy()
final_df = final_df.dropna()

print("Final dataset shape:", final_df.shape)
print("\nClass distribution:")
print(final_df['y'].value_counts())

# Save to CSV in the exact format you need
final_df.to_csv('amazon_stock_price.csv', index=False, header=False)
print("\nDataset saved as 'amazon_stock_price.csv'")

# Show sample of the final data
print("\nSample of the saved data:")
print(final_df.head(10).to_string(index=False))# Feature Engineering
df = data.copy()

# 1. Daily Return (x1): (Close - Open) / Open
df['x1'] = (df['Close'] - df['Open']) / df['Open'] * 100  # as percentage

# 2. Intraday Volatility (x2): (High - Low) / Open
df['x2'] = (df['High'] - df['Low']) / df['Open'] * 100  # as percentage

# 3. Volume Signal (x3): Volume / 30-day average volume
df['x3'] = df['Volume'] / df['Volume'].rolling(window=30).mean()

# 4. Price Trend (x4): 10-day MA / 30-day MA (momentum indicator)
df['ma_10'] = df['Close'].rolling(window=10).mean()
df['ma_30'] = df['Close'].rolling(window=30).mean()
df['x4'] = df['ma_10'] / df['ma_30']

# Create target variable (next day's movement)
threshold = 1.0  # 1% threshold for meaningful movement
df['next_day_return'] = df['Close'].pct_change().shift(-1) * 100

df['y'] = 'sideways'  # default
df.loc[df['next_day_return'] > threshold, 'y'] = 'upwards'
df.loc[df['next_day_return'] < -threshold, 'y'] = 'downwards'

# Keep only the columns we need and drop NaN values
final_df = df[['x1', 'x2', 'x3', 'x4', 'y']].copy()
final_df = final_df.dropna()

print("Final dataset shape:", final_df.shape)
print("\nClass distribution:")
print(final_df['y'].value_counts())

# Save to CSV in the exact format you need
final_df.to_csv('amazon_stock_price.csv', index=False, header=False)
print("\nDataset saved as 'amazon_stock_price.csv'")

# Show sample of the final data
print("\nSample of the saved data:")
print(final_df.head(10).to_string(index=False))

Final dataset shape: (3493, 5)

Class distribution:
y
sideways     1709
upwards       985
downwards     799
Name: count, dtype: int64

Dataset saved as 'amazon_stock_price.csv'

Sample of the saved data:
       x1       x2       x3       x4         y
                                              
-2.107274 2.765282 0.707773 0.951663 downwards
-0.649186 1.349620 0.706223 0.954718   upwards
 1.933702 3.185424 0.771826 0.958190  sideways
-0.330765 1.772543 0.560496 0.963326  sideways
 0.545277 2.377100 0.542223 0.966991  sideways
-0.652484 2.321839 0.567164 0.971651   upwards
 1.492036 2.246519 0.595190 0.975879 downwards
 0.025381 2.107130 0.766930 0.979040  sideways
 0.441127 2.061419 0.466541 0.980539   upwards
 4.919971 6.006734 1.073920 0.985358  sideways
Final dataset shape: (3493, 5)

Class distribution:
y
sideways     1709
upwards       985
downwards     799
Name: count, dtype: int64

Dataset saved as 'amazon_stock_price.csv'

Sample of the saved data:
       x1       x2       x3 

In [4]:
# Loading dataset and preparing data
df = pd.read_csv('./amazon_stock_price.csv', index_col=None, header=None) # change the path to your own path
df.columns = ['x1', 'x2', 'x3', 'x4', 'y']

d = {'upwards': 1,
     'downwards': 2,
     'sideways': 0,
}

df['y'] = df['y'].map(d)

# Assign features and target

X = torch.tensor(df[['x1', 'x2', 'x3', 'x4']].values, dtype=torch.float)
y = torch.tensor(df['y'].values, dtype=torch.int)

# Shuffling & train/test split

torch.manual_seed(123)
shuffle_idx = torch.randperm(y.size(0), dtype=torch.long)

X, y = X[shuffle_idx], y[shuffle_idx]

percent80 = int(shuffle_idx.size(0)*0.8)

X_train, X_test = X[shuffle_idx[:percent80]], X[shuffle_idx[percent80:]]
y_train, y_test = y[shuffle_idx[:percent80]], y[shuffle_idx[percent80:]]

# Normalize (mean zero, unit variance)

mu, sigma = X_train.mean(dim=0), X_train.std(dim=0)
X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma

### 1)  Logistic Regression for Multiclass Classification

In [5]:
torch.manual_seed(123) # for reproducibility 

class FeedforwardNeuralNetwork(nn.Module): # define class
    def __init__(self, num_features, hidden_size, num_classes): # initialze neural network
        super(FeedforwardNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(num_features, hidden_size) # create first layer to transform input data using weights and biases
        self.fc2 = nn.Linear(hidden_size, num_classes) # create second layer to produce final predictions
    
    def forward(self, x):
        x = F.relu(self.fc1(x)) # using the function from above - "rectified linear unit" transforming the input using the weights and biases of the first layer
        x = torch.sigmoid(self.fc2(x)) # logistic sigmoid function squeezes the input values to a range between 0 and 1
        return x

# Initializing the model   
num_features = 4 # give the task:'x1', 'x2', 'x3', 'x4'
hidden_size = 10 # can be any number (h)
num_classes = 3  # given the task:'upwards': 1,'downwards': 2,'sideways': 0
model = FeedforwardNeuralNetwork(num_features, hidden_size, num_classes)

# Initializing training parameters
num_epochs = 100 # can be any number (try to avoid over-/underfitting by choosing an appropriate number)
learning_rate = 1e-2 # can be any number (we choose this similar to the lecture)

criterion = nn.BCELoss() # binary cross entropy loss (BCE)
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # updates weights and biases during training in the first layer (fc1)

y_onehot = torch.zeros(y_train.size(0), num_classes) # initalizing a tensor with zeros (number of rows defined by y_train.size(0) and number of column by num_classes)
y_train_long = y_train.long().unsqueeze(1) # convert to long datatype and add an extra dimension of 1 to store the one_hot encoded labels
y_onehot.scatter_(1, y_train_long, 1) # fill one_hot encoded vector

# Training using binary cross entropy loss for each output node
for epoch in range(num_epochs): # iterating num_epochs
    outputs = model(X_train) # output predictions for each epoch
    loss = criterion(outputs, y_onehot) # loss is calculated using BCE
    optimizer.zero_grad() # gradients are cleared from previous period
    loss.backward() # gradients are computed
    optimizer.step() # update
    
    if (epoch+1) % 10 == 0:
        print("Progress: " f'epoch: [{epoch+1}/{num_epochs}], loss: {loss.item():.4f}') #print output
    
# Testing
with torch.no_grad(): # disabling gradient computation
    outputs = model(X_test) # output predictions for test samples
    _, predicted = torch.max(outputs.data, 1) # return indices of max values along dimension 1
    correct_predicted = (predicted == y_test).sum().item() # compute number of correct predictions
    accuracy_ffnn = 100 * correct_predicted / len(y_test) # percentage of correct predictions divided by total number of test samples
    print(f'Accuracy of model 1: {accuracy_ffnn:.2f}%') # print accuracy

Progress: epoch: [10/100], loss: 0.6245
Progress: epoch: [20/100], loss: 0.6027
Progress: epoch: [30/100], loss: 0.5985
Progress: epoch: [40/100], loss: 0.5963
Progress: epoch: [50/100], loss: 0.5948
Progress: epoch: [60/100], loss: 0.5939
Progress: epoch: [70/100], loss: 0.5933
Progress: epoch: [80/100], loss: 0.5927
Progress: epoch: [90/100], loss: 0.5923
Progress: epoch: [100/100], loss: 0.5919
Accuracy of model 1: 50.21%


### 2)  Softmax Regression with custom implementation of Cross Entropy Loss

In [6]:
torch.manual_seed(123) # for reproducibility 

class SoftmaxRegression(nn.Module):
    def __init__(self, num_features, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.fc = nn.Linear(num_features, num_classes)
    
    def forward(self, x):
        logits = self.fc(x)
        return logits

# One-hot encoding function
def one_hot_encoding(labels, num_classes):
    labels_long = labels.long().unsqueeze(1)
    return torch.zeros(len(labels), num_classes).scatter_(1, labels_long, 1)

# Softmax function
def softmax(logits):
    exp_logits = torch.exp(logits)
    sum_exp_logits = exp_logits.sum(dim=1, keepdim=True)
    probabilities = exp_logits / sum_exp_logits
    return probabilities # all continous probabilities add up to 1

# Cross-entropy loss function
def cross_entropy_loss(probabilities, one_hot_labels):
    log_probs = torch.log(probabilities)
    loss = -torch.sum(log_probs * one_hot_labels) / probabilities.shape[0]
    return loss

# Initializing the model
num_features = 4 
num_classes = 3
model = SoftmaxRegression(num_features, num_classes)

# Initializing training parameters
num_epochs = 100
learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training
for epoch in range(num_epochs):
    logits = model(X_train)
    probabilities = softmax(logits)
    one_hot_labels = one_hot_encoding(y_train, num_classes)
    loss = cross_entropy_loss(probabilities, one_hot_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print("Progress: " f'epoch [{epoch+1}/{num_epochs}], loss: {loss.item():.4f}')

# Testing
with torch.no_grad():
    logits = model(X_test)
    probabilities = softmax(logits)
    _, predicted = torch.max(probabilities, 1)
    correct = (predicted == y_test).sum().item()
    accuracy_sm_cel = 100 * correct / len(y_test)
    print(f'Accuracy of model 2: {accuracy_sm_cel:.2f}%')

Progress: epoch [10/100], loss: 1.0899
Progress: epoch [20/100], loss: 1.0568
Progress: epoch [30/100], loss: 1.0395
Progress: epoch [40/100], loss: 1.0308
Progress: epoch [50/100], loss: 1.0266
Progress: epoch [60/100], loss: 1.0248
Progress: epoch [70/100], loss: 1.0242
Progress: epoch [80/100], loss: 1.0241
Progress: epoch [90/100], loss: 1.0240
Progress: epoch [100/100], loss: 1.0240
Accuracy of model 2: 50.50%


### 3)  Softmax Regression with torch.nn.functional.nll_loss

In [7]:
torch.manual_seed(123)  # for reproducibility

class SoftmaxRegression(nn.Module):
    def __init__(self, num_features, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.fc = nn.Linear(num_features, num_classes)
    
    def forward(self, x):
        logits = self.fc(x)
        return F.log_softmax(logits, dim=1)  # log softmax 

# Initializing model
num_features = 4
num_classes = 3
model = SoftmaxRegression(num_features, num_classes)

# Initializing training parameters
num_epochs = 100
learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training
for epoch in range(num_epochs):
    log_probabilities = model(X_train) 
    loss = F.nll_loss(log_probabilities, y_train.long()) 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print("Progress: " f'epoch [{epoch+1}/{num_epochs}], loss: {loss.item():.4f}')

# Testing
with torch.no_grad():
    log_probabilities = model(X_test) 
    _, predicted = torch.max(log_probabilities, 1)  
    correct = (predicted == y_test).sum().item()
    accuracy_sm_tn = 100 * correct / len(y_test)
    print(f'Accuracy of model 3: {accuracy_sm_tn:.2f}%')

Progress: epoch [10/100], loss: 1.0899
Progress: epoch [20/100], loss: 1.0568
Progress: epoch [30/100], loss: 1.0395
Progress: epoch [40/100], loss: 1.0308
Progress: epoch [50/100], loss: 1.0266
Progress: epoch [60/100], loss: 1.0248
Progress: epoch [70/100], loss: 1.0242
Progress: epoch [80/100], loss: 1.0241
Progress: epoch [90/100], loss: 1.0240
Progress: epoch [100/100], loss: 1.0240
Accuracy of model 3: 50.50%


### 4)  Softmax Regression with torch.nn.functional.cross_entropy

In [8]:
torch.manual_seed(123)  # for reproducibility

class SoftmaxRegression(nn.Module):
    def __init__(self, num_features, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.fc = nn.Linear(num_features, num_classes)
    
    def forward(self, x):
        logits = self.fc(x)
        return logits  # no softmax here, since it's included in cross_entropy

# Initializing the model
num_features = 4
num_classes = 3
model = SoftmaxRegression(num_features, num_classes)

# Training parameters
num_epochs = 100
learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training 
for epoch in range(num_epochs):
    logits = model(X_train)  
    loss = torch.nn.functional.cross_entropy(logits, y_train.long()) 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print("Progress: " f'epoch [{epoch+1}/{num_epochs}], loss: {loss.item():.4f}')

# Testing
with torch.no_grad():
    logits = model(X_test) 
    _, predicted = torch.max(logits, 1)  
    correct = (predicted == y_test).sum().item()
    accuracy_sm_tnce = 100 * correct / len(y_test)
    print(f'Accuracy of model 4: {accuracy_sm_tnce:.2f}%')

Progress: epoch [10/100], loss: 1.0899
Progress: epoch [20/100], loss: 1.0568
Progress: epoch [30/100], loss: 1.0395
Progress: epoch [40/100], loss: 1.0308
Progress: epoch [50/100], loss: 1.0266
Progress: epoch [60/100], loss: 1.0248
Progress: epoch [70/100], loss: 1.0242
Progress: epoch [80/100], loss: 1.0241
Progress: epoch [90/100], loss: 1.0240
Progress: epoch [100/100], loss: 1.0240
Accuracy of model 4: 50.50%


### 5)  Softmax Regression with Mean Squared Error Loss

In [9]:
torch.manual_seed(123)  # for reproducibility

class SoftmaxRegression(nn.Module):
    def __init__(self, num_features, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.fc = nn.Linear(num_features, num_classes)
    
    def forward(self, x):
        logits = self.fc(x)
        return F.softmax(logits, dim=1)  # apply softmax 

# Initialize model
num_features = 4
num_classes = 3
model = SoftmaxRegression(num_features, num_classes)

# Initialize training parameters
num_epochs = 100
learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# One-hot encoding function
def one_hot_encoding(labels, num_classes):
    labels_long = labels.long().unsqueeze(1)
    return torch.zeros(len(labels), num_classes).scatter_(1, labels_long, 1)

# MSE Loss function
def mse_loss(predictions, one_hot_labels):
    return torch.mean((predictions - one_hot_labels) ** 2)

# Train
for epoch in range(num_epochs):
    probabilities = model(X_train)
    one_hot_labels = one_hot_encoding(y_train, num_classes)
    loss = mse_loss(probabilities, one_hot_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print("Progress: " f'epoch [{epoch+1}/{num_epochs}], loss: {loss.item():.4f}')

# Test
with torch.no_grad():
    probabilities = model(X_test)
    _, predicted = torch.max(probabilities, 1)
    correct = (predicted == y_test).sum().item()
    accuracy_sm_mse = 100 * correct / len(y_test)
    print(f'Accuracy of model 5: {accuracy_sm_mse:.2f}%')

Progress: epoch [10/100], loss: 0.2189
Progress: epoch [20/100], loss: 0.2111
Progress: epoch [30/100], loss: 0.2077
Progress: epoch [40/100], loss: 0.2059
Progress: epoch [50/100], loss: 0.2051
Progress: epoch [60/100], loss: 0.2047
Progress: epoch [70/100], loss: 0.2045
Progress: epoch [80/100], loss: 0.2045
Progress: epoch [90/100], loss: 0.2045
Progress: epoch [100/100], loss: 0.2045
Accuracy of model 5: 50.07%


### 6)  Linear Regression with Mean Squared Error Loss

In [10]:
torch.manual_seed(123)  # for reproducibility

class LinearRegression(nn.Module):
    def __init__(self, num_features):
        super(LinearRegression, self).__init__()
        self.fc = nn.Linear(num_features, 1) 
    
    def forward(self, x):
        output = self.fc(x)
        return output  # no activation function, direct regression

# Initializing the model
num_features = 4
model = LinearRegression(num_features)

# Initializing training parameters
num_epochs = 100
learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# MSE Loss function
criterion = nn.MSELoss()

# Training
for epoch in range(num_epochs):
    continuous_output = model(X_train).squeeze()  
    loss = criterion(continuous_output, y_train.float())  
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print("Progress: " f'epoch [{epoch+1}/{num_epochs}], loss: {loss.item():.4f}')

# Testing
with torch.no_grad():
    continuous_output = model(X_test).squeeze() 
    predicted_continuous = continuous_output.round()  
    predicted = predicted_continuous.long() 
    correct = (predicted == y_test).sum().item()
    accuracy_lr_mse = 100 * correct / len(y_test)
    print(f'Accuracy of model 6: {accuracy_lr_mse:.2f}%')

Progress: epoch [10/100], loss: 1.8658
Progress: epoch [20/100], loss: 1.6058
Progress: epoch [30/100], loss: 1.4252
Progress: epoch [40/100], loss: 1.2698
Progress: epoch [50/100], loss: 1.1360
Progress: epoch [60/100], loss: 1.0268
Progress: epoch [70/100], loss: 0.9372
Progress: epoch [80/100], loss: 0.8646
Progress: epoch [90/100], loss: 0.8067
Progress: epoch [100/100], loss: 0.7611
Accuracy of model 6: 49.50%


In [11]:
from termcolor import colored

In [12]:
# Define a function to print in color
def print_colored(text, color):
    print(colored(text, color))

# Header
header = "{:<60} {:<15}".format("Model Description", "Accuracy")
print_colored(header, "blue")
print_colored("-" * 75, "blue")

# Data
data = [
    ("Logistic Regression for Multiclass Classification", accuracy_ffnn),
    ("Softmax Regression with custom implementation of Cross Entropy Loss", accuracy_sm_cel)]
data

Model Description                                            Accuracy       
---------------------------------------------------------------------------


[('Logistic Regression for Multiclass Classification', 50.21459227467811),
 ('Softmax Regression with custom implementation of Cross Entropy Loss',
  50.50071530758226)]

In [13]:
#Comparison of model accuracy
print(f'Accuracy of model 1: Logistic Regression for Multiclass Classification: {accuracy_ffnn:.2f}%')
print(f'Accuracy of model 2: Softmax Regression with custom implementation of Cross Entropy Loss: {accuracy_sm_cel:.2f}%')
print(f'Accuracy of model 3: Softmax Regression with torch.nn.functional.nll_loss: {accuracy_sm_tn:.2f}%')
print(f'Accuracy of model 4: Softmax Regression with torch.nn.functional.cross_entropy: {accuracy_sm_tnce:.2f}%')
print(f'Accuracy of model 5: Softmax Regression with Mean Squared Error Loss:  {accuracy_sm_mse:.2f}%')
print(f'Accuracy of model 6: Linear Regression with Mean Squared Error Loss: {accuracy_lr_mse:.2f}%')


Accuracy of model 1: Logistic Regression for Multiclass Classification: 50.21%
Accuracy of model 2: Softmax Regression with custom implementation of Cross Entropy Loss: 50.50%
Accuracy of model 3: Softmax Regression with torch.nn.functional.nll_loss: 50.50%
Accuracy of model 4: Softmax Regression with torch.nn.functional.cross_entropy: 50.50%
Accuracy of model 5: Softmax Regression with Mean Squared Error Loss:  50.07%
Accuracy of model 6: Linear Regression with Mean Squared Error Loss: 49.50%
